<a href="https://colab.research.google.com/github/jaeheek03/git-practice/blob/master/MusicGen_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install MusicGen (from MusicGen Gradio Demo Colab Notebook)

In [ ]:
# @title Execute first for installing AudioCraft
# @markdown Execute this cell to install AudioCraft. (**Note:** the `pip install` stage might take a while.)
from pathlib import Path
import subprocess as sp
from shutil import rmtree
import re
import time

def run_log_on_fail(name, *args, **kwargs):
    log = log_folder / (name + ".log")
    print("Running stage", name)
    try:
        sp.run(*args, **kwargs, stdout=open(log, 'w'), stderr=sp.STDOUT, check=True)
    except sp.CalledProcessError:
        print(f"Stage {name} failed, this is NOT expected, see logs hereafter.")
        print(open(log, 'r').read())
        raise
    else:
        return log

def install():
    global log_folder
    ac_folder = Path('/content/audiocraft')
    log_folder = Path('/content/logs')
    log_folder.mkdir(exist_ok=True, parents=True)
    if ac_folder.exists():
        rmtree(ac_folder)
    try:
        run_log_on_fail("git clone", ["git", "clone", "https://github.com/facebookresearch/audiocraft"], cwd=ac_folder.parent)
        run_log_on_fail("pip install", ["pip", "install", "-e", "."], cwd=ac_folder)
        pass
    except sp.CalledProcessError:
        print("INSTALLATION FAILED...")
        return

install()

Running stage git clone
Running stage pip install


Load pre-trained MusicGen model

In [ ]:
from audiocraft.models import MusicGen
model = MusicGen.get_pretrained('facebook/musicgen-melody')


ModuleNotFoundError: No module named 'audiocraft.models'

Set generation parameters

In [ ]:
model.set_generation_params(duration=8)  # Adjust duration as needed.


Generate audio representations

In [ ]:
wav = model.generate_unconditional(4)  # Adjust the number of samples as needed.


In [ ]:
pip install -U datasets
pip install git+https://github.com/huggingface/transformers.git@main

In [ ]:
from datasets import load_dataset, Audio
from transformers import EncodecModel, AutoProcessor

# dummy dataset, however you can swap this with an dataset on the 🤗 hub or bring your own
librispeech_dummy = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")

# load the model + processor (for pre-processing the audio)
model = EncodecModel.from_pretrained("facebook/encodec_24khz")
processor = AutoProcessor.from_pretrained("facebook/encodec_24khz")

# cast the audio data to the correct sampling rate for the model
librispeech_dummy = librispeech_dummy.cast_column("audio", Audio(sampling_rate=processor.sampling_rate))
audio_sample = librispeech_dummy[0]["audio"]["array"]

# pre-process the inputs
inputs = processor(raw_audio=audio_sample, sampling_rate=processor.sampling_rate, return_tensors="pt")

# explicitly encode then decode the audio inputs
encoder_outputs = model.encode(inputs["input_values"], inputs["padding_mask"])
audio_values = model.decode(encoder_outputs.audio_codes, encoder_outputs.audio_scales, inputs["padding_mask"])[0]

# or the equivalent with a forward pass
audio_values = model(inputs["input_values"], inputs["padding_mask"]).audio_values

# you can also extract the discrete codebook representation for LM tasks
# output: concatenated tensor of all the representations
audio_codes = model(inputs["input_values"], inputs["padding_mask"]).audio_codes


In [ ]:
pip install -U encodec  # stable release
pip install -U git+https://git@github.com/facebookresearch/encodec#egg=encodec  # bleeding edge
# of if you cloned the repo locally
pip install .